# Export annotations from Prodigy

In [3]:
from collections import defaultdict
import copy
import json
import sys

import pandas as pd

sys.path.append("../..")
from src.data_prep.split_dataset import train_dev_test_split
from src.data_prep.prodigy_to_conll import get_bio_tagged_spans_from_jsonl, write_conll

In [4]:
db_file = "/Users/Carol/Documents/epicurious-recipes-with-rating-and-nutrition/labeled_data/food_ner_gold_2.jsonl"

## Count records and see if they're complete

In [10]:
records = defaultdict(list)
with open(db_file, 'r') as infile:
    for line in infile:
        record = json.loads(line)
        num = record['meta']['row']
        part = record['meta']['subpart']
        records[num].append(part)

In [11]:
# compare to input data
orig_file = "/Users/Carol/Documents/epicurious-recipes-with-rating-and-nutrition/use_directions.jsonl"
orig_records = defaultdict(list)
with open(db_file, 'r') as infile:
    for line in infile:
        record = json.loads(line)
        num = record['meta']['row']
        part = record['meta']['subpart']
        orig_records[num].append(part)

In [12]:
for rec in records:
    assert orig_records[rec] == records[rec]

In [13]:
# count labeled records
recipes = 0
sentences = 0
for rec in records:
    recipes += 1
    sentences += len(records[rec])
print(recipes)
print(sentences)

301
981


# Convert to CoNLL format

In [5]:
token_spans = get_bio_tagged_spans_from_jsonl(db_file)

In [6]:
len(token_spans)

981

In [7]:
# write the whole dataset out to conll 
outfile = "/Users/Carol/Documents/epicurious-recipes-with-rating-and-nutrition/food_gold.conll"
write_conll(token_spans, outfile)

In [36]:
# split into training, dev, and test sets
train, dev, test = train_dev_test_split(token_spans, 0.6, 0.2)

In [38]:
print(len(train), len(dev), len(test))

588 196 197


In [39]:
# write out training, dev, and test sets to conll
outfilestem = "/Users/Carol/Documents/epicurious-recipes-with-rating-and-nutrition/food_gold_{}.conll"
for dataset, name in zip([train, dev, test], ["train", "dev", "test"]):
    write_conll(dataset, outfilestem.format(name))   

# To import back to Prodigy for visualization

prodigy db-in [dataset] [in_file] [--loader] [--answer] [--overwrite] [--dry]
